# Objetivo

En este trabajo vamos a tratar de implementar el modelo de evaluación de riesgo de la epidemia desarrollado por el equipo de la [Computational Biology and Complex Systems; Universitat Politècnica de Catalunya - BarcelonaTech](https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf) . Para obtener los datos epidemiológicos nos apoyamos en el grupo de trabajo [#escovid19data](https://github.com/montera34/escovid19data).


@author: Manuel H. Arias 

@Twitter: @walyt

@mail: mharias@me.com



## Importación de librerias estándares para DAE

In [27]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import urllib.request

import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

## Descargamos la información

Hacemos en primer lugar una actualización de parámetros y preparación de variables que necesitaremos durante el ejercicio

In [28]:
pd.options.display.max_rows = 999 #Variable de contexto para permitir la presentación de datos por pantalla
pd.set_option('display.max_columns', None)

In [29]:
#url de este notebook en github
github_url = 'https://github.com/mharias/covid/blob/master/graficos_escovid19data.ipynb'

In [30]:
hashtag_fuente='#escovid19data' #hashtag de la fuente para los créditos

In [31]:
#url de la fuente de datos
path_montera34='https://github.com/montera34/escovid19data/blob/master/data/output/covid19-provincias-spain_consolidated.csv?raw=true'

Leemos los datos en un `pandas`

In [59]:
df = pd.read_csv(path_montera34)

veamos una rápida descripción de la información:

In [60]:
df.describe()

,ine_code,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_avg7,num_casos_prueba_pcr_avg7,ia14
count,16869.000000,5029.000000,4381.000000,1084.000000,1749.000000,11343.000000,11206.000000,12449.000000,7949.000000,9140.000000,7962.000000,16796.000000,16796.000000,16796.000000,16796.000000,16796.00000,1.686900e+04,7949.000000,11206.000000,12044.000000,11343.000000,6990.000000,7430.000000,8435.000000,8755.000000,9932.000000,9565.000000,9854.000000,9490.000000,12387.000000,11666.0,11633.000000,12085.000000,16869.000000,16484.000000,16484.000000,9911.000000
mean,26.499970,86.408431,133.556555,10366.536900,1233.436821,195.464692,27.702124,626.683830,7719.058121,10843.807549,2329.892346,89.888009,82.031377,0.274530,7.464873,0.11723,9.042216e+05,715.353804,3.045358,57.879337,22.604643,1699.479971,852.015343,1780.825726,887.842147,107.881091,107.765008,122.613179,121.335669,4.049891,inf,4.090957,3.963682,20.444958,90.897895,83.074654,199.865382
std,15.010492,167.936173,313.220676,26984.491392,1650.239251,744.564782,87.961274,1858.540522,18374.103678,32063.847915,3750.991723,287.973617,269.655552,1.534231,51.896609,1.37563,1.176267e+06,881.750952,3.890321,54.319017,40.088264,3773.278507,1902.921256,4922.331338,2459.116467,263.937677,244.861917,370.937365,338.573934,19.292239,NaN,19.439394,18.871876,112.723586,278.677277,260.371334,253.122488
min,1.000000,-67.000000,-67.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,8.477700e+04,0.000000,0.000000,0.000000,0.000000,-54.000000,-82.000000,-62.000000,-99.000000,-106.000000,-12.000000,-84.000000,-12.700000,-22.000000,-30.8,-1.300000,-0.600000,-4.000000,0.000000,0.000000,-6.900000
25%,13.000000,5.000000,7.000000,3.000000,200.000000,14.000000,2.000000,97.000000,873.000000,1572.750000,478.000000,1.000000,1.000000,0.000000,0.000000,0.00000,3.167980e+05,148.110000,0.410000,13.957500,3.030000,159.000000,70.000000,55.000000,26.000000,5.000000,7.000000,3.000000,3.700000,0.000000,0.0,0.000000,0.100000,0.000000,1.100000,1.100000,13.100000
50%,26.000000,31.000000,46.000000,21.000000,610.000000,55.000000,8.000000,220.000000,2463.000000,3503.500000,1330.500000,11.000000,11.000000,0.000000,0.000000,0.00000,5.810780e+05,393.880000,1.610000,37.530000,11.050000,610.000000,292.000000,422.000000,211.000000,32.000000,37.400000,25.000000,29.400000,0.000000,0.0,0.700000,0.700000,2.000000,12.950000,11.900000,97.400000
75%,40.000000,85.000000,139.000000,629.000000,1609.000000,155.000000,25.000000,415.000000,6184.000000,7088.500000,2625.750000,75.000000,70.000000,0.000000,0.000000,0.00000,1.032983e+06,924.760000,4.350000,93.260000,29.065000,1479.750000,739.750000,1436.000000,727.500000,98.000000,100.300000,102.000000,101.300000,3.000000,1.1,2.700000,2.700000,12.000000,78.100000,73.900000,303.150000
max,52.000000,1992.000000,4418.000000,121117.000000,14617.000000,15227.000000,1528.000000,13117.000000,224981.000000,346448.000000,45549.000000,6733.000000,6720.000000,32.000000,1190.000000,65.00000,6.663394e+06,5506.190000,28.720000,253.950000,474.750000,46102.000000,24458.000000,54308.000000,29115.000000,4423.000000,3494.000000,7013.000000,4159.300000,377.000000,inf,349.300000,338.400000,2369.000000,4181.400000,4171.300000,1485.300000


y un muestreo de valores y de algunas columnas de interés:

In [61]:
df.head()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_avg7,num_casos_prueba_pcr_avg7,ia14,source_name,source,comments
0,2020-01-01,Albacete,2,Castilla - La Mancha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,388167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
1,2020-01-01,Alicante/Alacant,3,Comunitat Valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1858683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
2,2020-01-01,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
3,2020-01-01,Araba/Álava,1,País Vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,331549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
4,2020-01-01,Asturias,33,"Asturias, Principado de",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1022800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN


In [62]:
df.tail()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_avg7,num_casos_prueba_pcr_avg7,ia14,source_name,source,comments
16864,2020-11-20,Sevilla,41,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,756.0,51855.0,NaN,18657.0,NaN,NaN,NaN,NaN,NaN,1942389,2669.65,NaN,38.92,NaN,12336.0,5239.0,NaN,NaN,859.0,748.4,NaN,NaN,8.0,1.1,12.3,11.1,78,NaN,NaN,635.1,Junta de Andalucía,https://www.juntadeandalucia.es/institutodeest...,NaN
16865,2020-11-20,Soria,42,Castilla y León,38.0,NaN,NaN,NaN,38.0,2.0,158.0,NaN,5342.0,643.0,NaN,NaN,NaN,NaN,NaN,88636,NaN,2.26,178.26,42.87,NaN,NaN,714.0,336.0,38.0,48.0,38.0,48.0,1.0,0.6,1.7,1.4,10,NaN,NaN,805.5,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
16866,2020-11-20,Valencia/València,46,Comunitat Valenciana,905.0,NaN,NaN,NaN,764.0,154.0,1107.0,49373.0,NaN,45549.0,NaN,NaN,NaN,NaN,NaN,2565124,1924.78,6.00,43.16,29.78,11654.0,5312.0,NaN,NaN,905.0,907.7,NaN,NaN,13.0,1.2,12.3,12.6,74,NaN,NaN,454.3,Sanidad Generalitat Valenciana,https://www.gva.es/es/inicio/area_de_prensa/no...,NaN
16867,2020-11-20,Valladolid,47,Castilla y León,365.0,NaN,NaN,NaN,299.0,87.0,709.0,NaN,27698.0,3940.0,NaN,NaN,NaN,NaN,NaN,519546,NaN,16.75,136.47,57.55,NaN,NaN,5866.0,2657.0,365.0,379.0,366.0,379.6,4.0,0.6,6.7,7.0,49,NaN,NaN,1129.1,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
16868,2020-11-20,Zamora,49,Castilla y León,123.0,NaN,NaN,NaN,119.0,10.0,254.0,NaN,7502.0,1065.0,NaN,NaN,NaN,NaN,NaN,172539,NaN,5.80,147.21,68.97,NaN,NaN,1679.0,779.0,123.0,111.3,123.0,111.3,4.0,1.6,3.3,3.6,25,NaN,NaN,973.1,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN


In [63]:
df['province'].unique()

array(['Albacete', 'Alicante/Alacant', 'Almería', 'Araba/Álava',
       'Asturias', 'Ávila', 'Badajoz', 'Balears, Illes', 'Barcelona',
       'Bizkaia', 'Burgos', 'Cáceres', 'Cádiz', 'Cantabria',
       'Castellón/Castelló', 'Ceuta', 'Ciudad Real', 'Córdoba',
       'Coruña, A', 'Cuenca', 'Gipuzkoa', 'Girona', 'Granada',
       'Guadalajara', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Lugo', 'Madrid', 'Málaga', 'Melilla', 'Murcia', 'Navarra',
       'Ourense', 'Palencia', 'Palmas, Las', 'Pontevedra', 'Rioja, La',
       'Salamanca', 'Santa Cruz de Tenerife', 'Segovia', 'Sevilla',
       'Soria', 'Tarragona', 'Teruel', 'Toledo', 'Valencia/València',
       'Valladolid', 'Zamora', 'Zaragoza'], dtype=object)

Vamos a formatear correctamente la columna `Date`, para posteriormente ordenar el $pandas$ por fecha creciente:

In [64]:
df['date']=pd.to_datetime(df['date'])

In [65]:
df.columns

Index(['date', 'province', 'ine_code', 'ccaa', 'new_cases', 'PCR', 'TestAc',
       'activos', 'hospitalized', 'intensive_care', 'deceased',
       'cases_accumulated', 'cases_accumulated_PCR', 'recovered', 'num_casos',
       'num_casos_prueba_pcr', 'num_casos_prueba_test_ac',
       'num_casos_prueba_otras', 'num_casos_prueba_desconocida', 'poblacion',
       'cases_per_cienmil', 'intensive_care_per_1000000',
       'deceassed_per_100000', 'hospitalized_per_100000', 'cases_14days',
       'cases_7days', 'cases_PCR_14days', 'cases_PCR_7days', 'daily_cases',
       'daily_cases_avg7', 'daily_cases_PCR', 'daily_cases_PCR_avg7',
       'daily_deaths', 'daily_deaths_inc', 'daily_deaths_avg3',
       'daily_deaths_avg7', 'deaths_last_week', 'num_casos_avg7',
       'num_casos_prueba_pcr_avg7', 'ia14', 'source_name', 'source',
       'comments'],
      dtype='object')

# Gráfico de BiocomSC

## preparamos el Pandas

In [33]:
# Gráfico de BiocomSC


provincias=['Madrid']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=3,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='incidencia_acumulada_14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'daily_cases_PCR'
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
for provincia in provincias:
    
    datos.loc[datos['province']==provincia,AI]=\
    datos.loc[datos['province']==provincia,casos_PCR_14]\
    /datos.loc[datos['province']==provincia,'poblacion']*100000

    datos.loc[datos['province']==provincia,clave_ratio] = \
    datos.loc[datos['province']==provincia][AI].pct_change(periods=7).add(1)    
    
    datos.loc[datos['province']==provincia,clave_casos_popu]=\
    datos.loc[datos['province']==provincia,casos_nuevos]\
    /datos.loc[datos['province']==provincia,'poblacion']*1000000

#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=3,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),'daily_cases_PCR'].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),'daily_cases_PCR'].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

In [34]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
#y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
#x0, x1 = 0,300
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    #print (i)
    #ax.set_xlim(0,x1)
    #ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Madrid\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)


lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [35]:
len(datos.index)

236

In [36]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

In [37]:
anim.save('modelo_biocomsc_madrid.mp4',dpi=288)

# Gráfico de BiocomSC Badajoz

In [83]:
provincias=['Barcelona']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=7,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='ia14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'PCR' #Navarra
casos_nuevos = 'new_cases' #badajoz #ojo con elegir el campo correcto...
casos_nuevos = 'daily_cases' #Barcelona
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=7,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),casos_nuevos].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),casos_nuevos].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos[(datos['date']>=datetime(year=2020,month=8,day=1))]
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

In [84]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
x0, x1 = 0,1200
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    
    #print (i)
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Badajoz\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)
    

    
lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [85]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

<ipython-input-84-58ad3c867318>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-84-58ad3c867318>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-84-58ad3c867318>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,


In [82]:
anim.save('videos/modelo_biocomsc_barcelona.mp4',dpi=288)

<ipython-input-79-58ad3c867318>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-79-58ad3c867318>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-79-58ad3c867318>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,
